## The pancreatic $\beta$-cell

Pancreatic $\beta$-cells secrete insulin, a hormone used to maintain blood plasma glocose levels. Electrical activity and $Ca^{2+}$ concentrations have been found to be linked to insuling secretion.

The Chay-Keizer model is a system of coupled ODEs which govern the flow of ions through the cell membrane and can be used to model the electrical activity and ion concentration in pancreatic $\beta$-cells.

### Chay-Keizer model

This model used modified Morris-Lecar equations to model the rich phenomena found in pancreatic $\beta$-cells such as spiking and bursting behaviour.

$$
C_m\frac{dV}{dt} = -g_{Ca}m_{\infty}(V-V_{Ca}) - g_Kw(V-V_K) - g_L(V-V_L) - I_{K(Ca)} \\
\\
\frac{dw}{dt} = \frac{\phi (w_{\infty}- w)}{\tau} \\
\\
\frac{[Ca^{2+}]_i}{dt} = f_i(-\alpha g_{Ca}m_{\infty}(V-V_{Ca}) - v_{LPM}[Ca^{2+}]_i)

$$

The K(Ca) current from the 1st equation is of the form:

$$
I_{K(Ca)} = g_{K(Ca)}\frac{[Ca^{2+}]_i}{K_{K(Ca)}+ [Ca^{2+}]_i}(V-V_K)
$$

In [2]:
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import numpy as np
import matplotlib.pyplot as plt


In [3]:
Cm = 5300 #fF
g_Ca = 1000 #pS
V_Ca = 25 #mV
g_K = 2700 #pS
V_K = -75 #mV
I_app = 0
v1 = -20 #mV
v2 = 24
v3 = -16 #mV
v4 = 11.2 #mV
phi = 0.035 #/ms
g_L = 150 #pS
V_L = -75 #mV
g_KCa = 2000 #pS
K_KCa = 5 #micro M
f = 0.001 
alpha = 0.0000045 #micro M /(fA * ms)


f_i = 0.004
vLPM = 0.18 #/ms


Missing `I_Ca`, `I_K`, `I_L`

`f_i` is the fraction of free $[Ca^{+2}]_i$

`alpha` is $\alpha = 10^3/(2FVi)$

In [4]:
#m_inf = 0.5*(1 + np.tanh((V - v1)/v2))
#w_inf = 0.5*(1 + np.tanh((V - v3)/v4))
#tau = 1/np.cosh((V - v3)/(2*v4))



#I_KCa = g_KCa * Cai  * (V - V_K) / (K_KCa + Cai)

#dVdt = (-I_Ca - I_K - I_L - I_KCa) / Cm

#dCaidt = f_i *(-alpha* I_Ca - vLPM * Cai)

#dwdt = phi * (w_inf - w)/ tau

In [7]:
def m_inf(V, v1, v2):
    return 0.5*(1+ np.tanh((V - v1))/v2)

def w_inf(V, v3, v4):
    return 0.5*(1 + np.tanh((V - v3)/v4))

def tau(V, v3, v4):
    return 1/np.cosh((V - v3)/(2*v4))




def I_KCa(V, Cai, g_KCa, V_K, K_KCa):
    return g_KCa * Cai  * (V - V_K) / (K_KCa + Cai)


def I_Ca(V, g_Ca,V_Ca, v1=-20,v2=24):
    return g_Ca * (V - V_Ca) * 0.5*(1+ np.tanh((V - v1))/v2)

def I_K(V, g_K, V_K, w):
    return g_K * ( V - V_K) *w 

def I_L(V, g_L, V_L):
    return g_L *(V - V_L)



def chay_Keizer(t, x, ):
    V, w, Cai = x

    
    dxdt = [
        (- I_Ca(V, g_Ca,V_Ca) - I_K(V, g_K, V_K,w) - I_L(V, g_L, V_L) - I_KCa(V, Cai, g_KCa, V_K, K_KCa)) / Cm,
        phi * (0.5*(1 + np.tanh((V - v3)/v4)) - w)/ tau(V, v3, v4),
        f_i *(-alpha* I_Ca(V, g_Ca,V_Ca) - vLPM * Cai)
        
    ]
    return dxdt

In [8]:

f_i = 0.004

vLPM = 0.18 #/ms
alpha = 4.5e-06



sol=solve_ivp(chay_Keizer, t_span=[0, 2400], y0 =[-55, 0.2, 0.05], max_step = 0.01)

print(sol)




KeyboardInterrupt: 

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))




ax1.plot(sol.t, sol.y[0, :], label="V (mV)")
ax1.set_xlabel("t (ms)")
ax1.set_ylabel("V (mV)")
ax1.legend()

ax2.plot(sol.t, sol.y[1, :], label="w")
ax2.set_xlabel("t (ms)")
ax2.set_ylabel("w")
ax2.legend()

ax3.plot(sol.t, sol.y[2, :], label="Cai")
ax3.set_xlabel("t (ms)")
ax3.set_ylabel("Cai")
ax3.legend()

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined